In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [4]:
import time

def Sigma(n):
    ans=0
    for a in [1..n+1]:
        term=sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)


def recipSigma(n):
    g=L(1/Sigma(2*n))
    return g

def recipSigmaPowerConstant(n):
    f=recipSigma(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=5100)
import pickle
datafile= open('/Users/barrybrent/run10may23.txt','wb') 
data=[]
start=time.time()/60
for a in [1..5000]:
    c=recipSigmaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/60
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(1, 0.0007772482931613922)
(2, 0.004447396844625473)
(3, 0.01677798479795456)
(4, 0.026377297937870026)
(5, 0.0396009162068367)
(6, 0.05232023447751999)
(7, 0.07034171745181084)
(8, 0.08302135020494461)
(9, 0.10145339369773865)
(10, 0.11976631358265877)
(11, 0.1428881473839283)
(12, 0.15977013111114502)
(13, 0.18166176229715347)
(14, 0.20379897952079773)
(15, 0.22971641644835472)
(16, 0.24852099642157555)
(17, 0.26985153183341026)
(18, 0.2902115508913994)
(19, 0.31637896224856377)
(20, 0.33699462935328484)
(21, 0.3620641157031059)
(22, 0.386617548763752)
(23, 0.41550109535455704)
(24, 0.43524621799588203)
(25, 0.4595109485089779)
(26, 0.48381469398736954)
(27, 0.5131227634847164)
(28, 0.5383206978440285)
(29, 0.5679522305727005)
(30, 0.5975238308310509)
(31, 0.6304312124848366)
(32, 0.650566678494215)
(33, 0.6749417446553707)
(34, 0.6989810802042484)
(35, 0.7276103310286999)
(36, 0.7511904798448086)
(37, 0.7794461287558079)
(38, 0.8069142960011959)
(39, 0.8426291979849339)
(40, 0.86905

(327, 12.445011463016272)
(328, 12.48332042992115)
(329, 12.526025049388409)
(330, 12.568493865430355)
(331, 12.615575045347214)
(332, 12.657710712403059)
(333, 12.704824216663837)
(334, 12.751765228807926)
(335, 12.802722211927176)
(336, 12.840525846928358)
(337, 12.883181881159544)
(338, 12.925685580819845)
(339, 12.972345378249884)
(340, 13.014461547136307)
(341, 13.06126244738698)
(342, 13.107869263738394)
(343, 13.158818013966084)
(344, 13.201022282242775)
(345, 13.247918110340834)
(346, 13.294520478695631)
(347, 13.345227114856243)
(348, 13.391779147088528)
(349, 13.442379795014858)
(350, 13.493859130889177)
(351, 13.549498431384563)
(352, 13.587694495916367)
(353, 13.630324430763721)
(354, 13.672595746815205)
(355, 13.719528410583735)
(356, 13.761631708592176)
(357, 13.807909995317459)
(358, 13.854380082339048)
(359, 13.907412379980087)
(360, 13.95196794718504)
(361, 13.998975031077862)
(362, 14.045985650271177)
(363, 14.097014147788286)
(364, 14.143389832228422)
(365, 14.195435

(647, 28.28186959773302)
(648, 28.324012499302626)
(649, 28.370404448360205)
(650, 28.41681521385908)
(651, 28.468252416700125)
(652, 28.514089949429035)
(653, 28.564768064767122)
(654, 28.61554366722703)
(655, 28.669799331575632)
(656, 28.71249482780695)
(657, 28.758556343615055)
(658, 28.80493239685893)
(659, 28.855072896927595)
(660, 28.901180114597082)
(661, 28.95185798406601)
(662, 29.002337362617254)
(663, 29.056900177150965)
(664, 29.103349681943655)
(665, 29.153767079114914)
(666, 29.20396303385496)
(667, 29.25808149576187)
(668, 29.308027766644955)
(669, 29.36325228214264)
(670, 29.4188377186656)
(671, 29.47790591046214)
(672, 29.519696462899446)
(673, 29.566065695136786)
(674, 29.612426195293665)
(675, 29.664080765098333)
(676, 29.71211374923587)
(677, 29.7639181278646)
(678, 29.816034596413374)
(679, 29.872074116021395)
(680, 29.91986121237278)
(681, 29.972163032740355)
(682, 30.024092081934214)
(683, 30.08025386556983)
(684, 30.133434414863586)
(685, 30.18918591365218)
(686

(971, 46.28986957669258)
(972, 46.347498912364244)
(973, 46.40936782956123)
(974, 46.471494399011135)
(975, 46.53740306571126)
(976, 46.59135115146637)
(977, 46.64913270995021)
(978, 46.707065530121326)
(979, 46.77040959894657)
(980, 46.828255012631416)
(981, 46.890246994793415)
(982, 46.95216681435704)
(983, 47.01838469877839)
(984, 47.076618045568466)
(985, 47.138385616242886)
(986, 47.199987180531025)
(987, 47.26618691161275)
(988, 47.32815221324563)
(989, 47.394124779850245)
(990, 47.46009216085076)
(991, 47.5297889970243)
(992, 47.58367331326008)
(993, 47.642176765948534)
(994, 47.7006698474288)
(995, 47.763043846935034)
(996, 47.82042419910431)
(997, 47.882150929421186)
(998, 47.94414161518216)
(999, 48.01012882962823)
(1000, 48.068378396332264)
(1001, 48.13069859519601)
(1002, 48.1925384812057)
(1003, 48.25849626213312)
(1004, 48.320625949651)
(1005, 48.38646296411753)
(1006, 48.4537152312696)
(1007, 48.52394409477711)
(1008, 48.58238003030419)
(1009, 48.64457508176565)
(1010, 4

(1286, 65.07485013455153)
(1287, 65.13482063263655)
(1288, 65.1851834282279)
(1289, 65.23968436196446)
(1290, 65.29458714649081)
(1291, 65.35340327769518)
(1292, 65.40790029987693)
(1293, 65.46721961721778)
(1294, 65.5256428308785)
(1295, 65.58895906433463)
(1296, 65.63987026363611)
(1297, 65.6948603130877)
(1298, 65.74959649890661)
(1299, 65.8085714802146)
(1300, 65.86308326199651)
(1301, 65.9220557808876)
(1302, 65.9810492657125)
(1303, 66.04462748020887)
(1304, 66.0994948297739)
(1305, 66.15811667963862)
(1306, 66.21693098172545)
(1307, 66.28022226318717)
(1308, 66.33870451524854)
(1309, 66.40151721611619)
(1310, 66.46573486179113)
(1311, 66.53390341624618)
(1312, 66.58565121516585)
(1313, 66.64096554741263)
(1314, 66.6955884322524)
(1315, 66.7550596781075)
(1316, 66.80983619764447)
(1317, 66.8686067648232)
(1318, 66.92753668129444)
(1319, 66.99042852967978)
(1320, 67.04534804821014)
(1321, 67.10460721328855)
(1322, 67.16322342678905)
(1323, 67.22625011578202)
(1324, 67.285086460411

...

(3453, 218.09932511672378)
(3454, 218.18644703179598)
(3455, 218.2782190963626)
(3456, 218.34453132748604)
(3457, 218.41526201367378)
(3458, 218.48649862781167)
(3459, 218.56163968145847)
(3460, 218.63277081400156)
(3461, 218.70822824910283)
(3462, 218.7825504615903)
(3463, 218.8612839318812)
(3464, 218.9317567460239)
(3465, 219.00632551312447)
(3466, 219.08148743212223)
(3467, 219.16060451045632)
(3468, 219.23509968072176)
(3469, 219.31423074752092)
(3470, 219.3932476826012)
(3471, 219.47655311599374)
(3472, 219.54741891101003)
(3473, 219.62233823165298)
(3474, 219.69749549776316)
(3475, 219.77683437988162)
(3476, 219.85182942822576)
(3477, 219.93195059895515)
(3478, 220.0113711617887)
(3479, 220.09621061384678)
(3480, 220.17204483225942)
(3481, 220.25114894658327)
(3482, 220.33042637631297)
(3483, 220.41375636309385)
(3484, 220.49284844845533)
(3485, 220.57581379637122)
(3486, 220.65965451300144)
(3487, 220.74720737710595)
(3488, 220.8177385814488)
(3489, 220.89223804697394)
(3490, 2

(3761, 242.62103591486812)
(3762, 242.70292726159096)
(3763, 242.7889788672328)
(3764, 242.87049584835768)
(3765, 242.95602994784713)
(3766, 243.04187972843647)
(3767, 243.1312289610505)
(3768, 243.21273004636168)
(3769, 243.2979606129229)
(3770, 243.38351143151522)
(3771, 243.47384088113904)
(3772, 243.55979964509606)
(3773, 243.64983124658465)
(3774, 243.73982874676585)
(3775, 243.83323346823454)
(3776, 243.9058765657246)
(3777, 243.98274813219905)
(3778, 244.05982226505876)
(3779, 244.14092183485627)
(3780, 244.2184115462005)
(3781, 244.2996062450111)
(3782, 244.38127792999148)
(3783, 244.4666710793972)
(3784, 244.5439989939332)
(3785, 244.62666089460254)
(3786, 244.70956171303988)
(3787, 244.79576877877116)
(3788, 244.87782832607627)
(3789, 244.96303144469857)
(3790, 245.04908737912774)
(3791, 245.13957776501775)
(3792, 245.2166482321918)
(3793, 245.2982261478901)
(3794, 245.3796969950199)
(3795, 245.46586476638913)
(3796, 245.5485289618373)
(3797, 245.63476255908608)
(3798, 245.72

(4070, 269.352770332247)
(4071, 269.4497478492558)
(4072, 269.53819048032165)
(4073, 269.6313787288964)
(4074, 269.7322062328458)
(4075, 269.82904414460063)
(4076, 269.92185309529305)
(4077, 270.0186729133129)
(4078, 270.11703823134303)
(4079, 270.21825271472335)
(4080, 270.3069392628968)
(4081, 270.39953634515405)
(4082, 270.49236645922065)
(4083, 270.5898989997804)
(4084, 270.6841786131263)
(4085, 270.78149334713817)
(4086, 270.878728531301)
(4087, 270.9799597635865)
(4088, 271.0731819793582)
(4089, 271.1702811457217)
(4090, 271.2675059325993)
(4091, 271.3687052987516)
(4092, 271.4656503684819)
(4093, 271.566871099174)
(4094, 271.6692579127848)
(4095, 271.7753239944577)
(4096, 271.8338077291846)
(4097, 271.8980727493763)
(4098, 271.96356588229537)
(4099, 272.03261479735374)
(4100, 272.0979003831744)
(4101, 272.16753386706114)
(4102, 272.23644449934363)
(4103, 272.30992321297526)
(4104, 272.3753128796816)
(4105, 272.4446661658585)
(4106, 272.51364244520664)
(4107, 272.58813747763634)


(4382, 294.0487315990031)
(4383, 294.1350205950439)
(4384, 294.20435992628336)
(4385, 294.27811639755964)
(4386, 294.3516082316637)
(4387, 294.4294226951897)
(4388, 294.5035324469209)
(4389, 294.5817849636078)
(4390, 294.6611290127039)
(4391, 294.7447763644159)
(4392, 294.8195389099419)
(4393, 294.8976082801819)
(4394, 294.9767611287534)
(4395, 295.0594522841275)
(4396, 295.1386178173125)
(4397, 295.2207521162927)
(4398, 295.30346654728055)
(4399, 295.38973984494805)
(4400, 295.463881097734)
(4401, 295.5419082455337)
(4402, 295.62066519632936)
(4403, 295.70320706442)
(4404, 295.7817300967872)
(4405, 295.86379401013255)
(4406, 295.9463095329702)
(4407, 296.03264416381717)
(4408, 296.1109725981951)
(4409, 296.1931055150926)
(4410, 296.27534376457334)
(4411, 296.36165634542704)
(4412, 296.44401316717267)
(4413, 296.53007086366415)
(4414, 296.6168473958969)
(4415, 296.7086915485561)
(4416, 296.77913949638605)
(4417, 296.8541706316173)
(4418, 296.9285948649049)
(4419, 297.0068403631449)
(44

(4693, 319.7818376645446)
(4694, 319.86744333058596)
(4695, 319.9565385468304)
(4696, 320.03750913217664)
(4697, 320.1235831975937)
(4698, 320.20877446234226)
(4699, 320.2977276146412)
(4700, 320.3827684484422)
(4701, 320.4716846831143)
(4702, 320.5610594339669)
(4703, 320.65428641438484)
(4704, 320.7309255152941)
(4705, 320.8113132826984)
(4706, 320.892459012568)
(4707, 320.9777456484735)
(4708, 321.05897414684296)
(4709, 321.1443368829787)
(4710, 321.23060305044055)
(4711, 321.3204123303294)
(4712, 321.4016478769481)
(4713, 321.4866686761379)
(4714, 321.5714949145913)
(4715, 321.6604060307145)
(4716, 321.7454258315265)
(4717, 321.8358430787921)
(4718, 321.9250755123794)
(4719, 322.0185723602772)
(4720, 322.0998768620193)
(4721, 322.18463661521673)
(4722, 322.2703586816788)
(4723, 322.3595152311027)
(4724, 322.44494996219873)
(4725, 322.5340232811868)
(4726, 322.62321016564965)
(4727, 322.71671882644296)
(4728, 322.80152774602175)
(4729, 322.8910529501736)
(4730, 322.9801372997463)
(4

In [6]:
import time

def Sigma(n):
    ans=0
    for a in [1..n+1]:
        term=sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)


def recipSigma(n):
    g=L(1/Sigma(2*n))
    return g

def recipSigmaPowerConstant(n):
    f=recipSigma(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=10000)
import pickle
datafile= open('/Users/barrybrent/run10may23no2.txt','wb') 
data=[]
start=time.time()/60
for a in [5001..7000]:
    c=recipSigmaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/60
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(5001, 0.3487795852124691)
(5002, 0.7206687368452549)
(5003, 1.1229812875390053)
(5004, 1.4939178824424744)
(5005, 1.888552587479353)
(5006, 2.2527187690138817)
(5007, 2.6367827653884888)
(5008, 2.9573549032211304)
(5009, 3.300272986292839)
(5010, 3.6407065503299236)
(5011, 4.002879332751036)
(5012, 4.346188604831696)
(5013, 4.7070580907166)
(5014, 5.06751000508666)
(5015, 5.4492082670331)
(5016, 5.791691523045301)
(5017, 6.153101269155741)
(5018, 6.516406636685133)
(5019, 6.897621102631092)
(5020, 7.259633302688599)
(5021, 7.643978483974934)
(5022, 8.02620941773057)
(5023, 8.431615088135004)
(5024, 8.75316596776247)
(5025, 9.094940923154354)
(5026, 9.436109185218811)
(5027, 9.797682166099548)
(5028, 10.13809760287404)
(5029, 10.502964071929455)
(5030, 10.865640670061111)
(5031, 11.248928219079971)
(5032, 11.589483503252268)
(5033, 11.949840351939201)
(5034, 12.314957566559315)
(5035, 12.697605036199093)
(5036, 13.058148939162493)
(5037, 13.440700482577085)
(5038, 13.821972735226154)
(

(5314, 110.79413193464279)
(5315, 111.1636801995337)
(5316, 111.48826165124774)
(5317, 111.83121590316296)
(5318, 112.17448578774929)
(5319, 112.53783562034369)
(5320, 112.8613224029541)
(5321, 113.20440533384681)
(5322, 113.55399677157402)
(5323, 113.95276265591383)
(5324, 114.32442615181208)
(5325, 114.70738777145743)
(5326, 115.10206078737974)
(5327, 115.51523501425982)
(5328, 115.87490176782012)
(5329, 116.2501787878573)
(5330, 116.6133329346776)
(5331, 117.01222203671932)
(5332, 117.38495798781514)
(5333, 117.78807290270925)
(5334, 118.18524606898427)
(5335, 118.58550452068448)
(5336, 118.9302367195487)
(5337, 119.29497276991606)
(5338, 119.6582869514823)
(5339, 120.04135063663125)
(5340, 120.41311817243695)
(5341, 120.80195313692093)
(5342, 121.18485581502318)
(5343, 121.58754435181618)
(5344, 121.9095434397459)
(5345, 122.25368853658438)
(5346, 122.59746551886201)
(5347, 122.96016468852758)
(5348, 123.30255403742194)
(5349, 123.6679131872952)
(5350, 124.03449850156903)
(5351, 12

(5621, 223.35448361560702)
(5622, 223.7693395242095)
(5623, 224.2030978165567)
(5624, 224.5982383005321)
(5625, 225.01097410172224)
(5626, 225.42450988292694)
(5627, 225.8612774387002)
(5628, 226.27453015372157)
(5629, 226.7079819366336)
(5630, 227.14265838637948)
(5631, 227.59380478784442)
(5632, 227.8834210522473)
(5633, 228.1925423555076)
(5634, 228.50165746733546)
(5635, 228.83109421655536)
(5636, 229.14180848747492)
(5637, 229.47060960158706)
(5638, 229.79876685142517)
(5639, 230.14840937033296)
(5640, 230.45894572138786)
(5641, 230.7888684682548)
(5642, 231.1176369562745)
(5643, 231.46662866696715)
(5644, 231.7951664328575)
(5645, 232.14572278782725)
(5646, 232.49613475054502)
(5647, 232.8661480359733)
(5648, 233.17521053552628)
(5649, 233.505281470716)
(5650, 233.83504600450397)
(5651, 234.1858332529664)
(5652, 234.5149289853871)
(5653, 234.86448672041297)
(5654, 235.21340913325548)
(5655, 235.58283907175064)
(5656, 235.91369450092316)
(5657, 236.26346735656261)
(5658, 236.61222

(5931, 340.8556924685836)
(5932, 341.2271712869406)
(5933, 341.61807318776846)
(5934, 342.00943275541067)
(5935, 342.4231446571648)
(5936, 342.775936819613)
(5937, 343.14775348454714)
(5938, 343.51842553541064)
(5939, 343.9104406349361)
(5940, 344.28280410170555)
(5941, 344.67430118471384)
(5942, 345.0651398859918)
(5943, 345.4769923686981)
(5944, 345.84973886981606)
(5945, 346.2415451705456)
(5946, 346.6323426552117)
(5947, 347.0428536720574)
(5948, 347.4343468658626)
(5949, 347.8462483994663)
(5950, 348.2588004730642)
(5951, 348.6896053850651)
(5952, 349.02002719044685)
(5953, 349.3741233572364)
(5954, 349.7256326377392)
(5955, 350.0966541543603)
(5956, 350.44844860211015)
(5957, 350.82077572122216)
(5958, 351.1920876018703)
(5959, 351.58352890238166)
(5960, 351.93440115079284)
(5961, 352.30605325475335)
(5962, 352.6799854040146)
(5963, 353.07231210172176)
(5964, 353.44400580599904)
(5965, 353.8358640521765)
(5966, 354.23002603650093)
(5967, 354.64259772002697)
(5968, 354.99321774020

(6242, 459.0443542711437)
(6243, 459.40201934054494)
(6244, 459.7366697527468)
(6245, 460.0931611545384)
(6246, 460.4498335905373)
(6247, 460.82483230531216)
(6248, 461.1607617549598)
(6249, 461.51515178754926)
(6250, 461.870472535491)
(6251, 462.24629293754697)
(6252, 462.6039435341954)
(6253, 462.9763560369611)
(6254, 463.34858721867204)
(6255, 463.74128495529294)
(6256, 464.07400603219867)
(6257, 464.4266889542341)
(6258, 464.77862575650215)
(6259, 465.15192591771483)
(6260, 465.50381856784225)
(6261, 465.8781766034663)
(6262, 466.25077280402184)
(6263, 466.642942186445)
(6264, 466.9950896203518)
(6265, 467.3687768690288)
(6266, 467.74259673804045)
(6267, 468.13487720116973)
(6268, 468.5065840706229)
(6269, 468.89869048446417)
(6270, 469.29456278681755)
(6271, 469.7061784043908)
(6272, 469.99848321825266)
(6273, 470.31240847334266)
(6274, 470.62484277039766)
(6275, 470.9557437710464)
(6276, 471.26842868328094)
(6277, 471.60100208595395)
(6278, 471.93241680040956)
(6279, 472.28561596

(6556, 573.6321342512965)
(6557, 574.0279582850635)
(6558, 574.4266838058829)
(6559, 574.8433275558054)
(6560, 575.1865482218564)
(6561, 575.5423767864704)
(6562, 575.9060415029526)
(6563, 576.2944271676242)
(6564, 576.6504257358611)
(6565, 577.0317932069302)
(6566, 577.4081922844052)
(6567, 577.8067080378532)
(6568, 578.1626071855426)
(6569, 578.540563583374)
(6570, 578.9183135703206)
(6571, 579.3170341849327)
(6572, 579.6969429701567)
(6573, 580.0942875705659)
(6574, 580.4924300685525)
(6575, 580.9112899191678)
(6576, 581.2701503522694)
(6577, 581.6473802030087)
(6578, 582.0246743038297)
(6579, 582.4247522689402)
(6580, 582.8016725517809)
(6581, 583.1984979845583)
(6582, 583.5955830402672)
(6583, 584.0130716860294)
(6584, 584.390443585813)
(6585, 584.7875848710537)
(6586, 585.1841719523072)
(6587, 585.6019122377038)
(6588, 586.0016980394721)
(6589, 586.4193218536675)
(6590, 586.8369458019733)
(6591, 587.2764628864825)
(6592, 587.6129192560911)
(6593, 587.9678520746529)
(6594, 588.324

(6873, 693.3153306208551)
(6874, 693.7098862342536)
(6875, 694.1250392831862)
(6876, 694.5201198719442)
(6877, 694.9344009868801)
(6878, 695.3492012657225)
(6879, 695.7863709181547)
(6880, 696.1415744498372)
(6881, 696.5160870216787)
(6882, 696.8912631161511)
(6883, 697.2882880680263)
(6884, 697.6633165217936)
(6885, 698.0582145042717)
(6886, 698.4535303525627)
(6887, 698.867488604039)
(6888, 699.2440256886184)
(6889, 699.6391766704619)
(6890, 700.0337446369231)
(6891, 700.4529731348157)
(6892, 700.84928746894)
(6893, 701.2650156021118)
(6894, 701.6798760183156)
(6895, 702.1167857386172)
(6896, 702.4953367374837)
(6897, 702.8916349709034)
(6898, 703.2863419353962)
(6899, 703.7019166871905)
(6900, 704.0981134064496)
(6901, 704.5117680504918)
(6902, 704.9270461685956)
(6903, 705.3619672171772)
(6904, 705.7582100518048)
(6905, 706.1732449382544)
(6906, 706.5881213732064)
(6907, 707.0236014835536)
(6908, 707.4405303671956)
(6909, 707.8763669505715)
(6910, 708.3119635023177)
(6911, 708.7684